# Calculate molecular properties by PM7 method
- Requirements (free softwares)
    - ASE: Wrapper library to run mopac (https://wiki.fysik.dtu.dk/ase/about.html)
    - MOPAC: Package to calculate molecular orbitals (http://openmopac.net/)
    - Open Babel: Package to calculate molecular geometry (https://jcheminf.biomedcentral.com/articles/10.1186/1758-2946-3-33)

In [1]:
from ase.build import molecule
from ase.calculators.mopac import MOPAC
from ase.io import read
import numpy as np

In [2]:
#run MOPAC
"""
steps
1) read SMILES character from temp/smiles.txt
2) generate mol file by open babel
3) run PM7
4) return properties as dict
"""
def run_mopac():

    !babel -i smi "temp/smiles.txt" -o mol "temp/mol.mol" --gen3D

    prop_dict={}

    atoms = read("temp/mol.mol")
    atoms.calc = MOPAC()
    atoms.get_potential_energy()
    eigs = atoms.calc.get_eigenvalues()
    #somos = atoms.calc.get_somo_levels()
    homo, lumo = atoms.calc.get_homo_lumo_levels()
    a,b,c,=atoms.calc.get_dipole_moment()

    prop_dict["homo"]=homo
    prop_dict["lumo"]=lumo
    
    
    homo_ind=np.where(eigs==homo)[0]
    
    for name,param in zip(["homo-1","homo-2","homo-3",
                           "lumo+1","lumo-2","lumo+3"],
                          [homo_ind-1,homo_ind-2,homo_ind-3,
                           homo_ind+1+1,homo_ind+1+2,homo_ind+1+3]
                         ):
        try:
            prop_dict[name]=eigs[param][0]
        except:
            prop_dict[name]=np.nan

    prop_dict["dipole_x"]=a
    prop_dict["dipole_y"]=b
    prop_dict["dipole_z"]=c
    prop_dict["heat_formation"]=atoms.calc.get_final_heat_of_formation()
    
    try:
        prop_dict["magnetic_moment"]=atoms.calc.get_magnetic_moment()
    except:
        prop_dict["magnetic_moment"]=0
    return prop_dict

def write_smiles(smiles):
    with open("temp/smiles.txt", mode='w') as f:
        f.write(smiles)

In [3]:
#test code
smiles="C1CCCC1"
write_smiles(smiles)
run_mopac()

1 molecule converted
68 audit log messages 


{'homo': -11.08105,
 'lumo': 3.91795,
 'homo-1': -11.27516,
 'homo-2': -11.88507,
 'homo-3': -11.88754,
 'lumo+1': 3.9189,
 'lumo-2': 4.3086,
 'lumo+3': 5.09509,
 'dipole_x': 0.007703190373711153,
 'dipole_y': -0.002290137678670883,
 'dipole_z': 0.0006245830032738773,
 'heat_formation': -0.7864505210590806,
 'magnetic_moment': 0}

In [4]:
#run all data
import pandas as pd
from tqdm import tqdm
df=pd.read_csv("../../database/small_db.csv")

prop_dict={}

for i in tqdm(df["SMILES"][:10]):
    write_smiles(i)
    prop_dict[i]=run_mopac()

  0%|          | 0/10 [00:00<?, ?it/s]

1 molecule converted
69 audit log messages 3 debugging messages 


 10%|█         | 1/10 [00:01<00:11,  1.26s/it]

1 molecule converted
71 audit log messages 4 debugging messages 


 20%|██        | 2/10 [00:02<00:10,  1.34s/it]

...

 80%|████████  | 8/10 [00:06<00:01,  1.94it/s]

1 molecule converted
112 audit log messages 


 90%|█████████ | 9/10 [00:06<00:00,  2.16it/s]

1 molecule converted
116 audit log messages 1 debugging messages 


100%|██████████| 10/10 [00:07<00:00,  1.43it/s]


In [5]:
prop_dict

{'CCCCCC': {'homo': -11.10114,
  'lumo': 3.96218,
  'homo-1': -11.27784,
  'homo-2': -11.4681,
  'homo-3': -11.64048,
  'lumo+1': 4.28826,
  'lumo-2': 4.41327,
  'lumo+3': 4.87742,
  'dipole_x': -0.0008327773376985031,
  'dipole_y': -0.0010409716721231288,
  'dipole_z': -0.0010409716721231288,
  'heat_formation': -1.6334442694200135,
  'magnetic_moment': 0},
 'CCCCCCC': {'homo': -11.08127,
  'lumo': 3.91881,
  'homo-1': -11.19203,
  'homo-2': -11.32067,
  'homo-3': -11.55199,
  'lumo+1': 4.18036,
  'lumo-2': 4.32165,
  'lumo+3': 4.64726,
  'dipole_x': -0.005621247029464895,
  'dipole_y': -0.003955692354067889,
  'dipole_z': -0.004996664026191019,
  'heat_formation': -1.8504087577479296,
  'magnetic_moment': 0},
 'CCCCCCCC': {'homo': -10.92425,
  'lumo': 3.93648,
  'homo-1': -11.01479,
  'homo-2': -11.30109,
  'homo-3': -11.39124,
  'lumo+1': 4.24225,
  'lumo-2': 4.31743,
  'lumo+3': 4.5557,
  'dipole_x': -0.00041638866884925155,
  'dipole_y': -0.002706526347520135,
  'dipole_z': -0.000